### Agent-Lab: Multi-Agent -> Researcher

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### XAI Grok Researcher

In [2]:
# Create Workflow
xai_agent = experiment_utils.create_xai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="grok-code-fast", api_key=os.getenv("XAI_API_KEY")
)
xai_research_agent = container.coordinator_planner_supervisor_agent()
xai_workflow_builder = xai_research_agent.get_workflow_builder(xai_agent["id"])
xai_workflow = xai_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="The researcher has a knowledge base prepared with the Book Art of War. What is the 'pinnacle of excellence' according to the book?",
    agent_id=xai_agent["id"],
)

inputs = xai_research_agent.get_input_params(message, schema="public")
config = xai_research_agent.get_config(xai_agent["id"])
result = xai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = xai_research_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user is inquiring about the concept of 'pinnacle of excellence' as described in the Book of Art of War, which is available in the researcher's knowledge base.",
  "title": "Extracting the Meaning of 'Pinnacle of Excellence' from The Art of War",
  "steps": [
    {
      "agent_name": "researcher",
      "title": "Search Knowledge Base for Relevant Passage",
      "description": "Use the internal knowledge base to locate and summarize the passage or concept referred to as the 'pinnacle of excellence' in The Art of War. Output a Markdown report with the relevant quotes and context."
    },
    {
      "agent_name": "reporter",
      "title": "Compile Final Report",
      "description": "Write a professional report summarizing the findings from the researcher, clearly stating what the 'pinnacle of excellence' means according to the book."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

### Problem Statement
What is the 'pinnacle of excellence' according to the book The Art of War?

### Knowledge Base Results
The knowledge base contains The Art of War by Sun Tzu. A key passage from Chapter 3 states: "To subdue the enemy without fighting is the acme of skill. To break his alliance without fighting is the pinnacle of excellence." This indicates that the pinnacle of excellence refers to achieving victory by disrupting the enemy's alliances or coalitions without engaging in direct combat.

### Conclusion
According to The Art of War, the pinnacle of excellence is to break the enemy's alliance without fighting. This strategy emphasizes strategic maneuvering and psychological tactics over brute force, allowing one to win by preventing the enemy from uniting effectively.

---
### Anthropic Researcher

In [6]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-opus-4-5", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_research_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_research_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [7]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="The researcher has a knowledge base prepared with the Book Art of War. What is the 'pinnacle of excellence' according to the book?",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_research_agent.get_input_params(message, schema="public")
config = anthropic_research_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_research_agent.format_response(result)

ValidationError: 1 validation error for TaskProgress
message_content
  Input should be a valid string [type=string_type, input_value=[], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

In [8]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user is inquiring about the concept of 'pinnacle of excellence' as described in the Book of Art of War, which is available in the researcher's knowledge base.",
  "title": "Extracting the Meaning of 'Pinnacle of Excellence' from The Art of War",
  "steps": [
    {
      "agent_name": "researcher",
      "title": "Search Knowledge Base for Relevant Passage",
      "description": "Use the internal knowledge base to locate and summarize the passage or concept referred to as the 'pinnacle of excellence' in The Art of War. Output a Markdown report with the relevant quotes and context."
    },
    {
      "agent_name": "reporter",
      "title": "Compile Final Report",
      "description": "Write a professional report summarizing the findings from the researcher, clearly stating what the 'pinnacle of excellence' means according to the book."
    }
  ]
}
```

In [9]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

### Problem Statement
What is the 'pinnacle of excellence' according to the book The Art of War?

### Knowledge Base Results
The knowledge base contains The Art of War by Sun Tzu. A key passage from Chapter 3 states: "To subdue the enemy without fighting is the acme of skill. To break his alliance without fighting is the pinnacle of excellence." This indicates that the pinnacle of excellence refers to achieving victory by disrupting the enemy's alliances or coalitions without engaging in direct combat.

### Conclusion
According to The Art of War, the pinnacle of excellence is to break the enemy's alliance without fighting. This strategy emphasizes strategic maneuvering and psychological tactics over brute force, allowing one to win by preventing the enemy from uniting effectively.

---
### OpenAI Researcher

In [10]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="gpt-5-nano", api_key=os.getenv("OPENAI_API_KEY")
)
openai_research_agent = container.coordinator_planner_supervisor_agent()
openai_workflow_builder = openai_research_agent.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)

In [11]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="The researcher has a knowledge base prepared with the Book Art of War. What is the 'pinnacle of excellence' according to the book?",
    agent_id=openai_agent["id"],
)

inputs = openai_research_agent.get_input_params(message, schema="public")
config = openai_research_agent.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_research_agent.format_response(result)

In [12]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user asks for the 'pinnacle of excellence' according to The Art of War. I will locate the exact phrasing and provide the commonly accepted meaning and variants, then present a concise final answer.",
  "title": "Pinnacle of Excellence in The Art of War",
  "steps": [
    {
      "agent_name": "researcher",
      "title": "Identify the exact phrasing and context of 'pinnacle of excellence' in The Art of War",
      "description": "From the prepared knowledge base, locate the line(s) that translate to the 'pinnacle of excellence'. Provide the exact wording (with common translations) and brief interpretation: the phrase is commonly rendered as 'The supreme art of war is to subdue the enemy without fighting' and is also expressed as 'The pinnacle of excellence is to win without fighting'. Note that translations vary by edition. Output a Markdown report summarizing findings."
    },
    {
      "agent_name": "reporter",
      "title": "Deliver final concise answer to user",
      "description": "Present the answer: The pinnacle of excellence in The Art of War is to win without fighting; include the variant translations and a brief note on translation differences. Provide a clean, direct statement and a source reference."
    }
  ]
}
```

In [13]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Executive Summary
- The pinnacle of excellence in The Art of War is described as achieving victory without fighting. The knowledge base notes that exact wording varies by edition and translator, with several common renderings cited.

Key Findings
- Core idea: The highest form of strategic excellence is to win without direct confrontation.
- Common renderings found in the knowledge base:
  - “The supreme art of war is to subdue the enemy without fighting.”
  - “The pinnacle of excellence is to win without fighting.”
  - Some editions reference: “The greatest victory is that which requires no battle.”
- Translation differences are edition- and translator-dependent; there is no single canonical phrasing in the provided material.

Detailed Analysis
- The concept emphasizes achieving victory without engaging in combat, reflecting a preference for strategic outcomes over direct confrontation.
- Variations in translation reflect different editions and translators rather than a shift in underlying meaning.

Conclusions and Recommendations
- Conclusion: The pinnacle of excellence is to win without fighting; the exact wording depends on edition/translator.
- Recommendations:
  - If a precise quotation is required, specify the edition or translator used in your knowledge base to determine the exact wording.
  - For general communication, you may cite one or two common renderings and note translation variability.
- Data note: All statements are based on the provided knowledge base content; if you need a single definitive quotation, please provide the edition/translator reference from your knowledge base.